In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df

,Код_группы,Год_Поступления,Пособие,Общежитие,Наличие_Матери,Наличие_Отца,Опекунство,Село,Иностранец,КодФакультета,Статус,male,female,birth_year_int,basis,language,country,region,city,parents_country
ind,,,,,,,,,,,,,,,,,,,,
0,16019,2015,0.0,0.0,0,0.0,0.0,0.0,0.0,41.0,3,0,1,1998,2,1,8,14,74,10
1,14895,2010,0.0,0.0,1,1.0,0.0,0.0,0.0,28.0,4,1,0,1992,4,0,8,14,74,10
2,20103,2017,0.0,0.0,1,1.0,0.0,0.0,0.0,41.0,4,0,1,1999,1,1,8,14,31,10
3,19838,2017,0.0,0.0,1,1.0,0.0,0.0,0.0,35.0,4,0,1,1994,0,1,8,14,1310,10
4,19803,2017,0.0,0.0,1,1.0,0.0,0.0,0.0,28.0,4,0,1,1995,0,1,8,14,1310,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13579,17593,2016,0.0,0.0,1,1.0,0.0,0.0,0.0,34.0,3,1,0,1998,2,1,8,14,70,10
13580,19433,2015,0.0,1.0,0,0.0,0.0,0.0,0.0,26.0,4,0,1,1997,2,2,8,14,1310,10
13581,18723,2016,0.0,0.0,1,1.0,0.0,0.0,0.0,41.0,4,1,0,1998,3,1,8,14,70,10


In [5]:
# Создание тренировочного и тестового датафремов

train, test = train_test_split(base_df, test_size=0.2)

In [11]:
train_input = train.drop('Статус', axis=1)
train_output = train['Статус']
train_output

ind
10553    4
841      3
3806     4
30       3
6119     3
        ..
9730     4
13025    4
8752     4
1639     4
10676    3
Name: Статус, Length: 10867, dtype: int64

In [ ]:
test_input = test.drop('Статус', axis=1)
test_output = test['Статус']